In [1]:
GYRO = False
SEQUENCE_LENGTH = 6
SEQUENCE_OVERLAP = 5
BATCH_SIZE = 10
EPOCHS = 50
MODEL_NAME = f"physical_or_stationary_epochs:{EPOCHS}_batch:{BATCH_SIZE}_gyro:{GYRO}_window:{SEQUENCE_LENGTH}_overlap:{SEQUENCE_OVERLAP}.keras"

TRAINING_SIZE = 0.8
DEV_SIZE = 0.1
TEST_SIZE = 0.1

In [2]:
import file_tagger
import math
import numpy as np
import pandas as pd
import sequence_genrator
from sklearn.model_selection import train_test_split

import tensorflow as tf
import tensorflow_decision_forests as tfdf



2023-11-06 13:10:52.527213: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


ModuleNotFoundError: No module named 'tensorflow_decision_forests'

In [ ]:
DATA_DIRECTORY = "./all_respeck"
PHYSICAL_ACTIVITIES = {
    "sitting&normal_breathing",
    "standing&normal_breathing",
    "lying_down_left&normal_breathing",
    "lying_down_right&normal_breathing",
    "lying_down_back&normal_breathing",
    "lying_down_stomach&normal_breathing",
    "walking&normal_breathing",
    "ascending_stairs&normal_breathing",
    "descending_stairs&normal_breathing",
    "shuffle_walking&normal_breathing",
    "running&normal_breathing",
    "misc_movements&normal_breathing"
}

In [ ]:
def getLabel(recordingType):
    return 0 if recordingType in PHYSICAL_ACTIVITIES else 1


def generateTrainingData(directory, sequenceLength, overlap):
    tagged_data = []

    csv_dictionary = file_tagger.tag_directory(directory)

    for recordingType in csv_dictionary:
        for csv_file in csv_dictionary[recordingType]:
            sequences = sequence_genrator.generate_sequences_from_file_without_gyroscope(directory + "/" + csv_file, sequenceLength, overlap)
            for sequence in sequences:
                tagged_data.append((getLabel(recordingType), sequence))
            

    print ("there are " + str(len(tagged_data)) + " tagged sequences in the dataset")    
    return tagged_data

In [ ]:
# split data into training, dev, and test sets
def train_dev_test_split(data, labels, dev_size, test_size, random_state=42):
    # Split the data into training and temporary (dev + test) sets
    train_data, temp_data, train_labels, temp_labels = train_test_split(data, labels, test_size=(dev_size + test_size), random_state=random_state)
    
    # Split the temporary data into dev and test sets
    dev_data, test_data, dev_labels, test_labels = train_test_split(temp_data, temp_labels, 
                                                                 test_size=(test_size / (dev_size + test_size)), random_state=random_state)
    
    return train_data, dev_data, test_data, train_labels, dev_labels, test_labels

In [ ]:
def train_model_CNN(input_data, labels, unique_labels, epochs, batch_size, validation_data):
    # Define the CNN model for your specific input shape
    model = Sequential([
        layers.Conv1D(32, 3, activation='relu', input_shape=(75, 3)),
        layers.MaxPooling1D(2),
        layers.Conv1D(64, 3, activation='relu'),
        layers.Conv1D(64, 3, activation='relu'),
        layers.MaxPooling1D(2),
        layers.Conv1D(128, 3, activation='relu'),
        layers.Conv1D(128, 3, activation='relu'),
        layers.MaxPooling1D(2),
        layers.Dropout(0.5),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(64, activation='relu'),
        layers.Dense(len(unique_labels), activation='softmax')
    ])

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Train the CNN model
    model.fit(input_data, labels, epochs=epochs, batch_size=batch_size, validation_data=validation_data)


    return model

In [ ]:
# tagged_sequences = generateTrainingData(DATA_DIRECTORY, 3, 0)

# # Get lists of training data and labels
# sequences = [sequence for _, sequence in tagged_sequences]
# labels = [label for label, _ in tagged_sequences]


# # Convert to numpy arrays
# sequences = np.array(sequences, dtype=np.float32)
# labels = np.array(labels)

# train_data, dev_data, test_data, train_labels, dev_labels, test_labels = train_dev_test_split(sequences, labels, dev_size=DEV_SIZE, test_size=TEST_SIZE) #10% dev, 10% test


# model = train_model_CNN(train_data, train_labels, [0,1], epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(dev_data, dev_labels))

# test_loss, test_accuracy = model.evaluate(test_data, test_labels)
# print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

# # Save the trained model
# model.save(f"models/PhysicalStationaryBinaryClassifier/{MODEL_NAME}_{test_accuracy}")

there are 20160 tagged sequences in the dataset
Epoch 1/50
1613/1613 [==============================] - 14s 7ms/step - loss: 0.4540 - accuracy: 0.8213 - val_loss: 0.4257 - val_accuracy: 0.8338
Epoch 2/50
1613/1613 [==============================] - 12s 7ms/step - loss: 0.4084 - accuracy: 0.8470 - val_loss: 0.4171 - val_accuracy: 0.8398
Epoch 3/50
1613/1613 [==============================] - 12s 8ms/step - loss: 0.4021 - accuracy: 0.8501 - val_loss: 0.4172 - val_accuracy: 0.8398
Epoch 4/50
1613/1613 [==============================] - 13s 8ms/step - loss: 0.3473 - accuracy: 0.8527 - val_loss: 0.3309 - val_accuracy: 0.8413
Epoch 5/50
1613/1613 [==============================] - 12s 7ms/step - loss: 0.2905 - accuracy: 0.8610 - val_loss: 0.2854 - val_accuracy: 0.8537
Epoch 6/50
1613/1613 [==============================] - 11s 7ms/step - loss: 0.2805 - accuracy: 0.8653 - val_loss: 0.2894 - val_accuracy: 0.8576
Epoch 7/50
1613/1613 [==============================] - 12s 7ms/step - loss: 0.268

2023-11-05 02:57:57.862997: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,6,128]
	 [[{{node inputs}}]]
2023-11-05 02:57:58.433802: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,6,128]
	 [[{{node inputs}}]]


INFO:tensorflow:Assets written to: models/PhysicalStationaryBinaryClassifier/physical_or_stationary_epochs:50_batch:10_gyro:False_window:6_overlap:5.keras_0.898809552192688/assets


INFO:tensorflow:Assets written to: models/PhysicalStationaryBinaryClassifier/physical_or_stationary_epochs:50_batch:10_gyro:False_window:6_overlap:5.keras_0.898809552192688/assets
